## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [1]:
# Import libraries
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datetime

In [2]:
# Load and read data file.
income_df = pd.read_csv("store_income_data_example.csv")

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [3]:
# Select unique values in the 'country' column.
countries = income_df['country'].unique()
# View unique values in the 'country' column.
print(f"\n\033[1mThere are {len(countries)} unique countries\033[0m")
print(countries)

# Convert the column entries into lowercase.
income_df['country'] = income_df['country'].str.lower()

# Remove any trailing white spaces
income_df['country'] = income_df['country'].str.strip()

# View unique values in the 'country' column after cleaning.
countries = income_df['country'].unique()
print(f"\n\033[1mThere are {len(countries)} unique countries after cleaning\033[0m")
print(countries)


There are 34 unique countries
['UK ' 'united states of america' 'UNITED STATES' 'uk'
 ' United States of America' 'South Africa ' 'United States.'
 'United States' 'South Africa/' 'United States '
 'United States of America' 'South Africa.' 'United Kingdom '
 'United States of America ' 'United States of America/' 'south africa'
 'UK/' 'United Kingdom.' ' United Kingdom' ' South Africa'
 'United Kingdom/' 'SOUTH AFRICA' ' UK' 'united kingdom' 'UNITED KINGDOM'
 ' United States' 'UNITED STATES OF AMERICA' 'South Africa'
 'United States/' 'united states' 'United States of America.' 'UK'
 'United Kingdom' 'UK.']

There are 15 unique countries after cleaning
['uk' 'united states of america' 'united states' 'south africa'
 'united states.' 'south africa/' 'south africa.' 'united kingdom'
 'united states of america/' 'uk/' 'united kingdom.' 'united kingdom/'
 'united states/' 'united states of america.' 'uk.']


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [4]:
# Function to find and replace similar values with a standard one.
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # Extracts the unique values in the specified column.
    strings = df[column].unique()
    
    # Find the top 10 closest matches.
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Extracts only the matches with ratio of 90 or more.
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Identify rows with a close match.
    rows_with_matches = df[column].isin(close_matches)

    # Replaces those values with the standardized string.
    df.loc[rows_with_matches, column] = string_to_match
    
# Runs the above function to replace country names with standar strings.
replace_matches_in_column(df=income_df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=income_df, column='country', string_to_match="united states")
replace_matches_in_column(df=income_df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=income_df, column='country', string_to_match="south africa")
replace_matches_in_column(df=income_df, column='country', string_to_match="uk")

# Select unique values in the 'country' column after matching.
countries = income_df['country'].unique()

# View unique values in the 'country' column after matching.
print(f"\n\033[1mThere are {len(countries)} unique countries after matching.\033[0m")
print(countries)

# Standardize country names.
income_df["country"] = income_df["country"].replace({
    "united states of america": "United States",
    "uk": "united kingdom",
})

# Properly formats country names.
income_df['country'] = income_df['country'].apply(lambda x: x.title())

# Select unique values in the 'country' column after replacing.
countries = income_df['country'].unique()

# View unique values in the 'country' column after replacing.
print(f"\n\033[1mThere are {len(countries)} unique countries after replacing.\033[0m")
print(countries)


There are 5 unique countries after matching.
['uk' 'united states of america' 'united states' 'south africa'
 'united kingdom']

There are 3 unique countries after replacing.
['United Kingdom' 'United States' 'South Africa']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [5]:
# Convert 'date_measured' to datetime format
income_df["date_measured"] = pd.to_datetime(income_df["date_measured"], errors='coerce')

# Fetches the current date.
today = datetime.date.today()
# Calculate number of days between date in 'date_measured' and today's date.
income_df["days_ago"] = (today - income_df["date_measured"].dt.date).dt.days

# Display sample rows to verify cleaning
print("\n", income_df.head())


    id                   store_name         store_email  department  \
0   1   Cullen/Frost Bankers, Inc.                 NaN    Clothing   
1   2          Nordson Corporation                 NaN       Tools   
2   3        Stag Industrial, Inc.                 NaN      Beauty   
3   4          FIRST REPUBLIC BANK  ecanadine3@fc2.com  Automotive   
4   5  Mercantile Bank Corporation                 NaN        Baby   

         income date_measured         country  days_ago  
0  $54438554.24    2006-07-14  United Kingdom      6830  
1  $41744177.01    2006-12-03   United States      6688  
2  $36152340.34    2003-08-12   United States      7897  
3   $8928350.04    2006-10-26  United Kingdom      6726  
4  $33552742.32    1973-12-24  United Kingdom     18720  
